## Hybrid Search RAG

1) Create pinecone account
2) Generate API and save it in .env file
3) Run the script to create an index

In [7]:
# Importing the required libraries
from dotenv import load_dotenv
from langchain_community.retrievers import PineconeHybridSearchRetriever
import os
from pinecone import Pinecone,ServerlessSpec
from langchain_huggingface import HuggingFaceEmbeddings
load_dotenv()
os.environ["PINECONE_API_KEY"]=os.getenv("PINECONE_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [8]:
# Index name
index_name = "hybrid-search-langchain"
# Initialize pinecone client
pc=Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# Create a new index
if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name,
                    metric="dotproduct",
                    spec=ServerlessSpec(cloud="aws",region="us-east-1"),
                    dimension=384
                    )
    print(f"Index {index_name} created")

In [9]:
index=pc.Index(index_name)
index

In [10]:
# Create one for sparse and one for dense Matrix
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
from pinecone_text.sparse import BM25Encoder
bm25_enc=BM25Encoder().default()
bm25_enc

In [12]:
sentences=[
    "In 2025, I will visit to US.",
    "In 2025, I will pursue my Masters.",
    "In 2027, I aim to graduate."
]

# TFIDF values of sentences
bm25_enc.fit(sentences)
# store the values to json file
bm25_enc.dump("bm25_sen.json")

100%|██████████| 3/3 [00:00<00:00, 2970.47it/s]


In [13]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_enc,index=index)

In [14]:
retriever.add_texts(
    [
    "In 2025, I will visit to US.",
    "In 2025, I will pursue my Masters.",
    "In 2027, I aim to graduate."
]
)

100%|██████████| 1/1 [00:02<00:00,  2.01s/it]


In [15]:
retriever.invoke("What will i do in 2025?")

[Document(metadata={'score': 0.596325576}, page_content='In 2025, I will visit to US.'),
 Document(metadata={'score': 0.584058881}, page_content='In 2025, I will pursue my Masters.'),
 Document(metadata={'score': 0.351142764}, page_content='In 2027, I aim to graduate.')]